<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Архитектура фон Неймана](#toc1_)    
  - [Сегментация памяти](#toc1_1_)    
  - [Как выполняется программа?](#toc1_2_)    
  - [Ещё раз о линковке](#toc1_3_)    
  - [Стек вызовов](#toc1_4_)    
    - [Устройство стека](#toc1_4_1_)    
    - [Вызов функции](#toc1_4_2_)    
  - [Вызов функции (резюме)](#toc1_5_)    
  - [Переполнение стека](#toc1_6_)    
  - [Рекурсивные функции](#toc1_7_)    
- [Указатели и массивы](#toc2_)    
  - [Указатели](#toc2_1_)    
  - [Передача параметров по указателю](#toc2_2_)    
  - [Массивы](#toc2_3_)    
  - [Связь массивов и указателей](#toc2_4_)    
  - [Примеры](#toc2_5_)    
  - [Совет](#toc2_6_)    
  - [C-style строки.](#toc2_7_)    
- [Использование указателей](#toc3_)    
  - [Два способа передачи массива](#toc3_1_)    
  - [Возрат указателя/значения из функции](#toc3_2_)    
  - [Недостатки указателей](#toc3_3_)    
- [Ссылки](#toc4_)    
  - [Различия ссылок и указателей](#toc4_1_)    
  - [lvalue и rvalue](#toc4_2_)    
  - [Время жизни переменной](#toc4_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Архитектура фон Неймана](#toc0_)

Современных компьютеры построены по принципам архитектуры фон Неймана:
    
    1.Принцип однородности памяти. Команды и данные хранятся в одной и той же памяти и внешне в памяти неразличимы. 
        *) гарвардская архитектура (исторический артефакт) - данные отдельно
    2.Принцип адресности. Память состоит из пронумерованных ячеек.
    3.Принцип программного управления. Все вычисления представляются в виде последовательности команд.
    4.Принцип двоичного кодирования.Вся информация (данные и команды) кодируются двоичными числами.
        *) делались компьютеры с троичным кодированием

*) однородность памяти позволяет программе изменять свой собственный код. Так делать обычно не принято, то алгоритмы с таким подходом есть (например, алгоритм Брезенхема рисования линий).

## <a id='toc1_1_'></a>[Сегментация памяти](#toc0_)

∙Оперативная память, используемая в программе на C++, разделена на области двух типов:
    
    1. сегменты данных,
    2. сегменты кода (текстовые сегменты).

∙В сегментах кода содержится код программы (защищается средствами ОС от произвольного изменения).  
∙В сегментах данных располагаются данные программы (значения переменных, массивы и пр.).  
∙При запуске программы выделяются два сегмента данных:

    1. сегмент глобальных данных,
    2. стек (для локальных переменных/функций).

∙В процессе работы программы могут выделяться и освобождаться дополнительные сегменты памяти.  
∙Обращения к адресу вне выделенных сегментов — ошибка времени выполнения (access violation, segmentation fault).

## <a id='toc1_2_'></a>[Как выполняется программа?](#toc0_)

∙Каждой функции в скомпилированном коде соответствует отдельная секция.  
∙Адрес начала такой секции — это адрес функции.  
∙Телу функции соответствует последовательность команд процессора.  
∙Работа с данными происходит на уровне байт, информацияо типах отсутствует.  
∙В процессе выполнения адрес следующей инструкции хранится в специальном регистре процессора IP(Instruction Pointer).  
∙Команды выполняются последовательно, пока не встретится специальная команда (например, условный переход или вызов функции), которая изменит IP.

## <a id='toc1_3_'></a>[Ещё раз о линковке](#toc0_)

∙На этапе компиляции объектных файлов в места вызова функций подставляются имена функций.  
∙На этапе линковки в места вызова вместо имён функций подставляются их адреса.  
∙Ошибки линковки:

    1.undefined reference. Функция имеет объявление, но не имеет тела. 
    2.multiple definition. Функция имеет два или более определений.

∙Наиболее распространённый способ получить multiple definition — определить функцию в заголовочном файле, который включён в несколько.cpp файлов.

    $ g++ -c log2.cpp       # => log2.o
    $ objdump -d log2.o     # дизассемблерирование объектного файла
    $ objdump -x log2.o     # заголовочная информация из объектного файла (таблица символов, таблица релокации (ссылки на подключаемые функции для линковки) и др.)

## <a id='toc1_4_'></a>[Стек вызовов](#toc0_)

∙Стек вызовов — это сегмент данных, используемый для хранения локальных переменных и временных значений.  
∙Не стоит путать стек с одноимённой структурой данных, у стека в C++ можно обратиться к произвольной ячейке.  
∙Стек выделяется при запуске программы.  
∙Стек обычно небольшой по размеру (4Мб).  
∙Функции хранят свои локальные переменные на стеке.  
∙При выходе из функции соответствующая область стека объявляется свободной.  
∙Промежуточные значения, возникающие при вычислении сложных выражений, также хранятся на стеке.  

### <a id='toc1_4_1_'></a>[Устройство стека](#toc0_)

    void  bar( ) {
        int c;
    }

    void  foo( ) {
        int b = 3;
        bar ();
    }

    int  main( ) {
        int a = 3;
        foo ();
        bar ();
        return  0;
    }

В х86 стек вызовов растет вниз. На стеке со всеми локальными переменными main(), +foo(), +bar(), потом main(), +bar(), потом main().

### <a id='toc1_4_2_'></a>[Вызов функции](#toc0_)

    int  foo(int a, int b, bool c){
        double d = a * b * 2.71;
        int     h = c ? d : d / 2;
        return h;
    }
    
    int  main( ){
        int x = 1;
        int y = 2;
        x = foo (x, y, false);
        cout  << x;
        return  0;
    }

frame pointer - указывает на дно стека  
stack pointer - указывает на вершину стека  
На стеке сначала main(), х=1, потом y=2, потом скопированные значения аргументов: false, 2, 1 (аргументы складываются в обратном порядке и лежат, пока из не заберут).

    к ним добавляются ret_val (место, куда функция запишет свой возврат (если void, то не делается)), ret_addr (адрес возврата, место в коде, которое исполняется после вызова ф-ии), 
    registers (состояние основных предохранямыех регистров)
        далее заходим в функцию foo():
        frame pointer - переносится на вершину стека (для адресации локальных переменных функции), на стек идут d, h
            размеры ret_val, ret_addr, registers известны, поэтому аргументы функции адресуются обратно от frame pointer, а локальные переменные, очевидно, наоборот
            на стек кладутся вычисленные d = 5.42, h = 2
                переходим к возврату из foo()
            в ret_val заносится 2, регистры восстанавливаются, стек очищается выше ret_val
        последовательно снимается все что накоплено на стеке
            

## <a id='toc1_5_'></a>[Вызов функции (резюме)](#toc0_)

При вызове функции на стек складываются:

    1. aргументы функции,
    2. адрес возврата,
    3. значение frame pointer и регистров процессора.

Кроме этого на стеке резервируется место под возвращаемое значение.  
Параметры передаются в обратном порядке, что позволяет реализовать функции с переменным числом аргументов.  
Адресация локальных переменных функции и аргументов функции происходит относительно frame pointer.  
Конкретные детели процесса вызова зависит от используемых соглашений/компилятора/библиотек (cdecl, stdcall, fastcall, thiscall).

## <a id='toc1_6_'></a>[Переполнение стека](#toc0_)

Путем выхода за границы локального массива в функции можно в адрес возврата из функции на стеке вызовов положить произвольный адрес (из доступного сегмента памяти, естественно).  
Тогда при выходе из функции поломанный стек заставит выполнить код по указанному адресу.  
Можно также восстановить в ходе выполнения вредоносного кода адрес возврата исходной функции на стеке вызовов, и работа такого кода вообще пройдет бессимптомно.  


## <a id='toc1_7_'></a>[Рекурсивные функции](#toc0_)

С++ позволяет определять рекурсивные функции, т.е. функции которые вызывают сами себя напрямую или опосредованно. 
Факториал:

    int factorial(int n) {
        if (n == 0)
            return 1;
        return factorial(n - 1) * n;
    }

Число Фибоначчи с заданным номером:  

    int fibonacсi(int n) {
        if (n < 2)
            return n;
        return fibonacci(n - 1) + fibonacci(n - 2);
    }

Однако язык C++ не рассчитан для написания программ активно использующих рекурсию (в отличие от, например, некоторых функциональных языков программирования), поэтому важно следить, чтобы глубина рекурсии не была очень большой.

# <a id='toc2_'></a>[Указатели и массивы](#toc0_)

## <a id='toc2_1_'></a>[Указатели](#toc0_)

∙Указатель — это переменная, хранящая адрес некоторой ячейки памяти.  
∙Указатели являются типизированными.
    
    int   i = 3; // переменная типа int
    int * p = 0; // указатель на переменную типа int

∙Нулевому указателю (которому присвоено значение 0) не соответствует никакая ячейка памяти.  
∙Оператор взятия адреса переменной &.  
∙Оператор разыменования * (НЕ ПУТАТЬ С МОДИФИКАТОРОМ ТИПА).

    p  = &i; // указатель p указывает на переменную i
    *p = 10; // изменяется ячейка по адресу p, т.е. i

## <a id='toc2_2_'></a>[Передача параметров по указателю](#toc0_)

Рассмотрим функцию, меняющую параметры местами:

    void  swap (int a, int b) {
        int t = a; 
        a = b; 
        b = t;
    }
    
    int  main() {
        int k = 10, m = 20;
        swap (k, m);
        cout  << k << ’ ’ << m << endl; // 10 20
        return  0;
    }
Т. о. swap изменяет только свои локальные копии переменных k и m, а это не то что надо.  

Вместо значений типа int будем передавать указатели.  

    void  swap (int * a, int * b) {
        int t = *a;
        *a = *b;
        *b = t;
    }
    
    int  main() {
        int k = 10, m = 20;
        swap (&k, &m);
        cout  << k << ’ ’ << m << endl; // 20 10
        return  0;
    }

Теперь swap изменяет переменные k и m по указателям на них.  
\*) Передаваемые аргументы по прежнему копируются внутрь функции swap и работа идет с локальными копиями! Только на этот раз копируются указатели на интересующие нас переменные, а значить и их локальные копии тоже будут указывать на всё те же переменные!  

## <a id='toc2_3_'></a>[Массивы](#toc0_)

∙Массив — это набор однотипных элементов, расположенных в памяти друг за другом, доступ к которым осуществляется по индексу.  
∙C++ позволяет определять массивы непосредственно на стеке.  
    
    // массив 1 2 3 4 5 0 0 0 0 0
    int m[10] = {1, 2, 3, 4, 5};

∙Индексация массива начинается с 0, последний элемент массива длины n имеет индекс n - 1.
    
    for (int i = 0; i < 10; ++i)
        cout  << m[i] << ’ ’;
    cout  << endl;

## <a id='toc2_4_'></a>[Связь массивов и указателей](#toc0_)

∙Указатели позволяют передвигаться по массивам.  
∙Для этого используется арифметика указателей:

    int m[10] = {1, 2, 3, 4, 5};
    int * p = &m[0]; // адрес начала массива
    int * q = &m[9]; // адрес последнего элемента
    
        ∙(p + k)— сдвиг на k ячеек типа int вправо.
        ∙(p - k)— сдвиг на k ячеек типа int влево.
        ∙(q - p)— количество ячеек между указателями.
        ∙ p[k] эквивалентно *(p + k).

## <a id='toc2_5_'></a>[Примеры](#toc0_)

Заполнение массива:

    int m[10] = {}; // изначально заполнен нулями
    //             &m[0]         &m[9]
    for (int * p = m ; p <= m + 9; ++p )    // *p указывает на 1-й эл-т, ++p - сдвиг на размер одного эл-та
        *p = (p - m) + 1;                   // в адресуемую ячейку записывает (p - m) номер ячейки +1
    // Массив заполнен числами от 1 до 10

Передача массива в функцию:

    int  max_element (int * m, int  size) { // указатель на начало массива, размер массива
        int  max = *m;                      // инициализируем значением первого эл-та
        for (int i = 1; i < size; ++i)      
            if (m[i] > max)
                max = m[i];
        return  max;
    }

## <a id='toc2_6_'></a>[Совет](#toc0_)

У начинающих изучать C++ иногда возникает соблазн передать массив в функцию следующим образом:

    void foo(int a[3]) { /* ... */ }

Однако, не смотря на то, что такой код будет компилироваться, работает он не совсем так, как ожидается. Например:

    int a[1] = {};
    foo(a);

Хотя программист написавший функцию foo, вероятно, ожидает, что в нее можно будет передавать только массивы из трёх элементов, этот код успешно скомпилируется. Компилятор не увидит никаких проблем, так как размерность массива в определении функции проигнорируется. 

## <a id='toc2_7_'></a>[C-style строки.](#toc0_)

В языке C строки представляли как массивы char-ов, которые заканчиваются специальным символом ‘\0’ (на самом деле, это просто символ с номером 0). C++ сохраняет (в основном) совместимость с языком C, и поэтому поддерживает работу с C-style строками.  

Строковые литералы в С++ имеют тип массива, например, литерал “C-style string” имеет тип const char[15] — 14 символов строки и нулевой завершающий символ. Ключевое слово const в данном случае обозначает то, что изменять содержимое строки нельзя. Менять содержимое литералов — это довольно плохая затея. Для изменения строки, заданной строковым литералом, можно создать её копию в массиве:  

    char copy[15] = "C-style string";

Размер массива в таком определении указывать не обязательно — если массив инициализируется сразу при создании, то компилятор может и сам вычислить размер массива:

    char copy[] = "C-style string"; // размер массива 15 char-ов

# <a id='toc3_'></a>[Использование указателей](#toc0_)



## <a id='toc3_1_'></a>[Два способа передачи массива](#toc0_)

Функция для поиска элемента в массиве:
    
    bool contains(int * m, int size , int  value) {
        for (int i = 0; i != size; ++i)
            if (m[i] ==  value)                         // т.к. m[i] это *(m + i), тут еще арифметика, поэтому это медленнее
                return  true;
        return  false;
    }
    
    // указатели быстрее
    bool contains(int * p, int * q, int  value) {       // p начало массива, q следующий за последним эл. (чтобы можно было обработать и пустой массив)
        for (; p != q; ++p)
            if (*p ==  value)
                return  true;
        return  false;
    }

\*) ++i в цикле for предпочтительней, т.к. префиксная версия немного быстрее (не происходит сохранения текущего значения, возвращается сразу увеличенное)

## <a id='toc3_2_'></a>[Возрат указателя/значения из функции](#toc0_)

Функция для поиска максимума в массиве (ЗНАЧЕНИЕ):

    int  max_element (int * p, int * q) {
        int  max = *p;
        for (; p != q; ++p) 
            if (*p > max)
                max = *p;
        return  max;
    }
    
    int m[10] = {...};
    int  max = max_element(m, m + 10);
    cout  << "Maximum = " << max  << endl;

Функция для поиска максимума в массиве (УКАЗАТЕЛЬ НА ЗНАЧЕНИЕ):

    int * max_element (int * p, int * q) {
        int * pmax = p;
        for (; p != q; ++p)
            if (*p > *pmax)
                pmax = p;
        return  pmax;
    }
    
    int m[10] = {...};
    int * pmax = max_element(m, m + 10);
    cout  << "Maximum = " << *pmax  << endl;

Функция для поиска максимума в массиве (ЗНАЧЕНИЕ через УКАЗАТЕЛЬ):

    bool  max_element (int * p, int * q, int * res) {   // res - указатель на результат
        if (p == q)
            return  false;                              // если массив пустой
        *res = *p;                                      // инициализ. первым элементом массива
        for (; p != q; ++p)
            if (*p > *res)
                *res = *p;
        return  true;                                   // т.е. изменяется указатель-аргумент и еще возвращается флаг успех/неуспех
    }
    
    int m[10] = {...};
    int max = 0;
    if (max_element(m, m + 10, &max))                   // ссылка на int max
        cout  << "Maximum = " << max  << endl;

Функция для поиска максимума в массиве (через УКАЗАТЕЛЬ на УКАЗАТЕЛЬ (комбинация предыдущих)):

    bool  max_element (int * p, int * q, int ** res) {  // ** res - указатель на указатель на некотор.перем.
        if (p == q)
            return  false;
        *res = p;                                       // res ссылается на перем., в котор. лежит указатель на макс.элем.
        for (; p != q; ++p)
            if (*p > **res)                             // p пробегает элементы массива, а res ссылается на тот из них, кот. указ. на макс.
                *res = p;
        return  true;
    }
    
    int m[10] = {...};
    int * pmax = 0;                                     // указатель на некоторый int, иниц. 0, т.е. не указывает ни на что
    if (max_element(m, m + 10, &pmax))          
        cout  << "Maximum = " << *pmax  << endl;

Функции, которые вы реализовали в предыдущих заданиях, довольно полезны — их даже включили в стандартную библиотеку языков C и C++. В стандартной библиотеке они называются strlen, strcat и strstr (еще несколько незначительных отличий, но смысл такой же). Чтобы использовать их в C++ нужно подключить заголовок cstring:

    #include <cstring>
    char str[100] = "left part";
    strcat(str, " right part");
    size_t length = strlen(str);

Будьте внимательны: функция strcat не предоставляет никакого способа контроля за переполнением массива, это может служить источником ошибок. Вместо нее предпочитайте функцию strncat.

В стандартной библиотеке есть еще множество полезных функций. Старайтесь использовать уже готовые функции, когда возможно, вместо того, чтобы писать их самостоятельно.


В стандартной библиотеке C++ есть еще множество функций полезных при работе с массивами (и не только). Например, одной из самых частых задач с массивами является сортировка. Отсортировать массив чисел по возрастанию в C++ можно так:

    #include <algorithm> //здесь объявлена функция sort
    using namespace std;
    int a[100] = { ... };
    sort(a, a + 100);

Функция sort позволяет сортировать не только по возрастанию и не только числа. Строго говоря это не совсем функция, а шаблон функции. С шаблонами вы познакомитесь ближе к концу этого курса, однако это не мешает вам уже сейчас пользоваться такими стандартными функциями.

В заголовочном файле algorithm есть еще несколько полезных функций, которые вам могут потребоваться, например, функции поиска максимального и минимального элемента:

    int *minptr = min_element(a, a + 100);
    int *maxptr = max_element(a, a + 100);

Обратите внимание, что функции возвращают не сами элементы, а указатели на них.

Аналогичным образом вы можете использовать эти функции для массивов любых других типов, которые можно сравнивать с использованием оператора < (строки, например). Более подробно о стандартной библиотеке C++ будет рассказано в продолжении данного курса. 


## <a id='toc3_3_'></a>[Недостатки указателей](#toc0_)
 
∙Использование указателей синтаксически загрязняет код и усложняет его понимание. (Приходится использовать операторы * и &.)  
∙Указатели могут быть неинициализированными (некорректный код).  
∙Указатель может быть нулевым (корректный код), а значит указатель нужно проверять на равенство нулю.  
∙Арифметика указателей может сделать из корректногоуказателя некорректный (легко промахнуться).

# <a id='toc4_'></a>[Ссылки](#toc0_)

Для того, чтобы исправить некоторые недостатки указателей, в C++ введены ссылки.  
Ссылки являются “красивой обёрткой” над указателями

    void  swap (int & a, int & b) {     // ссылки на переменные (из типа делается ссылка)
        int t = b;
        b = a;                          // синтаксически - это локальные перем-е
        a = t;                          // на самом деле это (копии) ссылок на переменные
    }
    
    int  main() {
        int k = 10, m = 20;
        swap (k, m);
        cout  << k << ’ ’ << m << endl; // 20 10
        return  0;
    }


Например:

    int a = 100;
    int& b = a;         // b - это ссылка на а

    void test(int& a)   // а - внутри функции - является ссылкой на переданный функции аргумент, а не копией его значения
    {
    ...
    }

## <a id='toc4_1_'></a>[Различия ссылок и указателей](#toc0_)

∙Ссылка не может быть неинициализированной.

    int * p; // OK
    int & l; // ошибка

∙У ссылки нет нулевого значения.

    int * p = 0; // OK
    int & l = 0; // ошибка (ссылка не может никуда не указывать)

∙Ссылку нельзя переинициализировать. Нельзя переключить созданную ссылку на другую область памяти

    int a = 10, b = 20;
    int * p = &a;   // p указывает на a
    p = &b;         // p указывает на b
    int & l = a;    // l ссылается на a
    l = b;          // a присваивается значение b (а не переключение ссылки на b)

∙Нельзя получить адрес ссылки или ссылку на ссылку.

    int a = 10;
    int * p = &a;   // p указывает на a
    int ** pp = &p; // pp указывает на переменную p
    
    int & l = a;    // l ссылается на a
    int * pl = &l;  // pl указывает на переменную a
    int && ll = l;  // ошибка
    
∙Нельзя создавать массивы ссылок.

    int * mp[10] = {};   // массив указателей на int
    int & ml[10] = {};   // ошибка
    
∙Для ссылок нет арифметики. Ссылка реализует идею синонимов (псевдонимов), в отличии от указателей, поэтому нельзя определить что значит "следующий" в этом контексте.

## <a id='toc4_2_'></a>[lvalue и rvalue](#toc0_)

∙Выражения в C++ можно разделить на два типа:

    1.lvalue— выражения, значения которых являются ссылкой на переменную/элемент массива, а значит могут быть указаны слева от оператора =.
    2.rvalue— выражения, значения которых являются временными и не соответствуют никакой переменной/элементу массива.

*) 10 + 20 = 7 некорректное выражение именно в контексте этих понятий

∙Указатели и ссылки могут указывать только на lvalue.

    int a = 10, b = 20;
    int m[10] = {1,2,3,4,5,5,4,3,2,1};
    int & l1 = a;                       // OK
    int & l2 = a + b;                   // ошибка
    int & l3 = *(m + a / 2);            // OK (т.к. соотв.эл-ту массива m[a/2])
    int & l4 = *(m + a / 2) + 1;        // ошибка
    int & l5 = (a + b > 10) ? a : b;    // OK (ссылку будет либо на a, либо на b)

## <a id='toc4_3_'></a>[Время жизни переменной](#toc0_)

Следует следить за временем жизни переменных.

    int * foo() {       // возвр. указатель на локальную перем., которая не существует после завершения функции
        int a = 10;
        return &a;
    }
    
    int & bar() {       // возвр. ссылку на локальную перем., которая не существует после завершения функции
        int b = 20;
        return b;
    }
    
    int * p = foo();    // оба варианта некорректны
    int & l = bar();

В качестве параметров в функции могут передаваться не только ссылки на "обычные" переменные, но и ссылки на массивы. Следующая функция принимает ссылку на массив из трех значений типа int:

    void foo(int (&a)[3]) { /* ... */ }

Обратите внимание, что скобки вокруг параметра a в данной конструкции обязательны — ведь мы хотим передать ссылку на массив, а не массив ссылок.

Кажется, что такая конструкция не имеет особого смысла, ведь массивы при передаче в функцию не копируются. Однако, такая конструкция запрещает компилятору игнорировать размер массива. 

Например, следующий код не будет компилироваться:

    int a[1] = {};
    foo(a);

Компилятор g++ на это выдаст следующую ошибку:

error: invalid initialization of reference of type ‘int (&)[3]’ from expression of type ‘int [1]’

Т. е. компилятор не может преобразовать ссылку на массив из одного элемента в ссылку на массив из трех элементов (что и ожидалось).


Ссылки в C++ используются не только для передачи изменяемых параметров в функции, но и для передачи больших параметров, копировать которые дорого. Примерами значений, копирование которых может оказаться дорогой операцией, являются значения типа string, который вы уже могли видеть, или объекты классов-контейнеров: vector, list, map, set, с которыми вы еще не встречались.

В случае, если мы хотим избежать ненужного копирования при передаче параметров, но не хотим позволять изменять параметр внутри функции, его следует передавать по константной ссылке:

    int foo(string const &s) { /* нельзя менять значение s */ }

Для примитивных типов передача параметров по константной ссылке возможна, но не имеет особого смысла, т.к. их копирование дёшево.